<a href="https://colab.research.google.com/github/lmarieta/FaceRecognition/blob/main/Face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting, upload data.RData, data_submission.RData, IdLookupTable.csv files. After training your model, you can download it. After saving the output file, you can also download it.

Import Libraries


In [ ]:
import numpy as np
from tensorflow import keras
from keras import layers
import pandas as pd
import numpy as np
#from sklearn import train_test_split
import matplotlib.pyplot as plt
!pip install pyreadr
import pyreadr
from keras.optimizers import Adam
from timeit import default_timer
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.regularizers import l2

Load data

In [ ]:
filename = '/content/data.RData'
df = pyreadr.read_r(filename)

x_train = df['im.train']
y_train = df['d.train']
x_test = df['im.test']
y_test = df['d.test']

# Get the indices of rows containing NaN values. Note that train and test splits are random with a given seed.
nan_indices_train = x_train.index[x_train.isnull().any(axis=1)].tolist() + y_train.index[y_train.isnull().any(axis=1)].tolist()
nan_indices_test = x_test.index[x_test.isnull().any(axis=1)].tolist() + y_test.index[y_test.isnull().any(axis=1)].tolist()

# Drop rows containing NaN values
x_train = x_train.drop(x_train.index[nan_indices_train])
y_train = y_train.drop(y_train.index[nan_indices_train])
x_test = x_test.drop(x_test.index[nan_indices_test])
y_test = y_test.drop(y_test.index[nan_indices_test])



Model selection

In [ ]:
model_type = 'Convolutions' # options: {'Single hidden layer','Convolutions'}
df = pd.DataFrame({model_type}, columns=['Model type'])

# Start recording time
start = default_timer()

Single layer model

In [ ]:
if model_type == 'Single hidden layer' :
  pixel_num = x_train.shape[1]
  keras.backend.clear_session()
  # Define the model (single hidden layer)
  model = keras.Sequential([
      keras.layers.Dense(64, activation='relu', input_shape=(pixel_num,)),
      keras.layers.Dense(32, activation='relu'),
      keras.layers.Dense(y_train.shape[1])  # Output layer with 2 units per keypoint (x and y coordinates)
  ])
  learning_rate=0.05
  epsilon = 0.1
  optimizer = Adam(learning_rate=learning_rate,epsilon=epsilon)
  early_stopping = EarlyStopping(monitor='loss',patience=20, restore_best_weights=True)
  hyperparameters = {
    'optimizer': optimizer,
    'loss': 'mse',
    'learning_rate':learning_rate,
    'epsilon': epsilon,
    'early_stopping': early_stopping,
  }
  for k,v in hyperparameters.items():
    df[k] = v
  # Compile the model
  model.compile(optimizer=hyperparameters['optimizer'], loss=hyperparameters['loss'])

Convolutions model

In [ ]:
if model_type == 'Convolutions' :
  # Convolutional neural net with three convolutional layers and two fully connected layers
  keras.backend.clear_session()
  pixel_num = np.sqrt(x_train.shape[1]).astype(int)
  # Assuming you have defined the input shape of your images
  input_shape = (pixel_num, pixel_num, 1)

  # Create the CNN model
  model = keras.Sequential()

  # Convolutional Layer 1
  model.add(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D((3, 3)))

  # Convolutional Layer 2
  model.add(Conv2D(16, (3, 3), activation='relu',kernel_regularizer=l2(0.1)))
  model.add(MaxPooling2D((3, 3)))

  # Convolutional Layer 3
  model.add(Conv2D(16, (3, 3), activation='relu',kernel_regularizer=l2(0.1)))
  model.add(MaxPooling2D((3, 3)))

  # Flatten the feature maps
  model.add(Flatten())

  # Fully Connected Layer 1
  model.add(Dense(16, activation='relu',kernel_regularizer=l2(0.1)))

  # Fully Connected Layer 2
  model.add(Dense(16, activation='relu',kernel_regularizer=l2(0.1)))

  # Output Layer
  model.add(Dense(y_train.shape[1], activation='softmax'))

  # Compile the model
  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

  # Print the model summary
  model.summary()
  # stop early
  early_stopping = EarlyStopping(monitor='loss',patience=20, restore_best_weights=True)
  # reshape inputs
  x_train = x_train.to_numpy().reshape(-1, pixel_num, pixel_num, 1)
  x_test = x_test.to_numpy().reshape(-1, pixel_num, pixel_num, 1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 94, 94, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 9, 9, 16)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 16)          2320      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 16)         0

Analysis

In [ ]:
x_train.shape

(1738, 96, 96, 1)

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=400, batch_size=32, callbacks=[early_stopping],validation_data=(x_test, y_test))

# Extract the loss and validation loss values from the history object
loss_values = history.history['loss']
val_loss = history.history['val_loss']

# Convert loss_values to a DataFrame
df_loss = pd.DataFrame({'Epoch': np.arange(1, len(loss_values) + 1), 'Loss': loss_values})

# Make predictions
predictions = model.predict(x_test)

# Size of the image is given, prediction cannot be outside of range
min_value = 0
max_value = 96
constrained_predictions = np.clip(predictions, min_value, max_value)

# Save the model to a file
model.save('/content/simple_model.h5')

# Duration
duration = default_timer() - start
print(duration)
df['Duration'] = duration

Epoch 1/400
55/55 [==============================] - 24s 306ms/step - loss: 2619.0808 - accuracy: 0.6991 - val_loss: 2617.4636 - val_accuracy: 0.7065
Epoch 2/400
55/55 [==============================] - 14s 259ms/step - loss: 2616.8108 - accuracy: 0.7048 - val_loss: 2615.8958 - val_accuracy: 0.7065
Epoch 3/400
55/55 [==============================] - 9s 161ms/step - loss: 2615.6355 - accuracy: 0.7048 - val_loss: 2615.0422 - val_accuracy: 0.7065
Epoch 4/400
55/55 [==============================] - 7s 134ms/step - loss: 2614.9673 - accuracy: 0.7048 - val_loss: 2614.5295 - val_accuracy: 0.7065
Epoch 5/400
55/55 [==============================] - 9s 164ms/step - loss: 2614.5513 - accuracy: 0.7048 - val_loss: 2614.1985 - val_accuracy: 0.7065
Epoch 6/400
55/55 [==============================] - 8s 148ms/step - loss: 2614.2759 - accuracy: 0.7048 - val_loss: 2613.9746 - val_accuracy: 0.7065
Epoch 7/400
55/55 [==============================] - 8s 145ms/step - loss: 2614.0867 - accuracy: 0.7048 

KeyboardInterrupt: ignored

In [ ]:
# Plot the training and validation loss
plt.figure()
plt.plot(df_loss['Epoch'], loss_values, label='Training Loss')
plt.plot(df_loss['Epoch'], val_loss, label='Validation Loss')
plt.yscale('log')
plt.ylim(1,np.max(val_loss))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# First, calculate the column means along axis 0 (columns)
col_means = np.nanmean(y_test, axis=0)

# Next, create the matrix 'p' using the calculated column means
num_rows_d_test = y_test.shape[0]
num_cols_d_train = y_train.shape[1]

# Create the matrix 'p' with the specified dimensions and fill it with column means
p = np.tile(col_means, (num_rows_d_test, 1)).astype(y_test.dtypes)

# Compute RMSE
rmse = np.sqrt(df_loss['Loss'].iloc[-1])

# Print RMSE
print('RMSE : ')
print(rmse)
df['RMSE'] = rmse
df.to_csv('/content/performance.csv',mode='a')

In [ ]:
df_predictions = pd.DataFrame(predictions)

In [ ]:
# Plot the first image in the DataFrame
image = np.array(x_test.iloc[1,:])  # Convert the Series to a NumPy array
image = image.reshape((96, 96))  # Reshape the array to the desired image dimensions
predicted_keypoints = predictions[1].reshape((15, 2))
plt.scatter(predicted_keypoints[:, 0], predicted_keypoints[:, 1], c='r', marker='o')
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
from pandas.io.parsers.readers import read_csv
# Extract data for output
filename = '/content/data_submission.RData'
df_submission = pyreadr.read_r(filename)
x_submission = df_submission['im_submission']
# Get the indices of rows containing NaN values.
nan_indices_submission = x_submission.index[x_submission.isnull().any(axis=1)].tolist()
# Drop rows containing NaN values
x_submission = x_submission.drop(x_submission.index[nan_indices_submission])

# Make predictions
predictions_submission = model.predict(x_submission)

# Size of the image is given, prediction cannot be outside of range
min_value = 0
max_value = 96
constrained_predictions_submission = np.clip(predictions_submission, min_value, max_value)

# Remove predictions not requested in IdLookupTable.csv
filename = '/content/IdLookupTable.csv'
IdLookupTable = read_csv(filename)
# All keypoints names
keypoints_names = IdLookupTable['FeatureName'].iloc[0:30]
# Calculate the repeating sequence based on the number of rows in 'df'
repeating_sequence = np.repeat(np.arange(1, 1783*30 // 30 + 1), 30)
# Add the repeating sequence as a new column to the DataFrame
keypoints_names_filter = pd.DataFrame({'ImageId': repeating_sequence})
keypoints_names_filter['Indices'] = range(len(keypoints_names_filter))
keypoints_names_filter['FeatureName'] = pd.concat([keypoints_names] * 1783, ignore_index=True)
# Filter
keypoints_names_filter = keypoints_names_filter.merge(IdLookupTable[['ImageId', 'FeatureName']], on=['ImageId', 'FeatureName'], how='inner')
# Create a mask of size 1783x30 where True indicates the indices to keep
nb_output_images = x_submission.shape[0]
mask = np.isin(np.arange(nb_output_images*30), keypoints_names_filter['Indices'])
# Reshape the mask to the same shape as 'np_array'
mask = mask.reshape(constrained_predictions_submission.shape)
# Use the mask to replace values in the NumPy array with NaN
constrained_predictions_submission[~mask] = np.nan

# Plot the first image in the DataFrame
image = np.array(x_submission.iloc[-1,:])  # Convert the Series to a NumPy array
image = image.reshape((96, 96))  # Reshape the array to the desired image dimensions
predicted_keypoints = constrained_predictions_submission[-1].reshape((15, 2))
plt.scatter(predicted_keypoints[:, 0], predicted_keypoints[:, 1], c='r', marker='o')
plt.imshow(image, cmap='gray')
plt.show()

# Prepare for output file
df_predictions = pd.DataFrame(constrained_predictions_submission)

# Convert the DataFrame to a NumPy array and reshape it
reshaped_values = df_predictions.values.reshape(-1, 1)

# Remove NaN values from the NumPy array
reshaped_values = reshaped_values[~np.isnan(reshaped_values)]

# Create a new DataFrame with the reshaped values
reshaped_df = pd.DataFrame(reshaped_values, columns=['Value'])
indices = pd.DataFrame(range(1,len(reshaped_df)+1))
submission = pd.concat([indices, reshaped_df], axis=1)
submission.rename(columns={0: 'RowId','Value': 'Location'}, inplace=True)

# Write to csv file
submission.to_csv('/content/simple_model_submission.csv',index=False)
